In [3]:
import pandas as pd

data = pd.read_csv("textual_data_an1_cleaned_final.csv")

print(data.head())

                                   clean_tweet_2  label
0                                          nigga      4
1                                horses retarded      5
2  nigga momma youngboy spitting real shit nigga      0
3            rt xxsugvngxx ran holy nigga today       1
4                       everybody calling nigger      1


In [4]:
!pip install gensim

Defaulting to user installation because normal site-packages is not writeable


In [5]:
!pip install numpy==1.24.4

Defaulting to user installation because normal site-packages is not writeable


In [6]:
filtered_data = data.dropna(subset=['clean_tweet_2']).copy()

# Now extract both in sync
sentences = filtered_data['clean_tweet_2'].astype(str).tolist()
y = filtered_data['label'].astype(int).tolist()

# Optional: binarize the labels
y = [1 if label != 0 else 0 for label in y]

# Confirm
print("Number of sentences:", len(sentences))
print("Number of labels:", len(y))

Number of sentences: 149822
Number of labels: 149822


In [7]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
import nltk

# Verify if 'punkt' is available
nltk.download('punkt')

tokenized_sentences = [sentence.split() for sentence in sentences]
tokenizer = RegexpTokenizer(r'\w+')
tokenized_sentences = [tokenizer.tokenize(str(sentence).lower()) for sentence in sentences]

model = Word2Vec(sentences=tokenized_sentences, vector_size=100, window=5, min_count=1, sg=1, epochs=10)

model.save("word2vec_model.model")

[nltk_data] Downloading package punkt to /Users/admin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
print("Number of sentences:", len(sentences))
print("Number of labels:", len(y))

Number of sentences: 149822
Number of labels: 149822


In [9]:
model = Word2Vec.load('word2vec_model.model')

word_vector = model.wv['shit']
print("Vector for 'shit':", word_vector)

similar_words = model.wv.most_similar('shit')
print("Words similar to 'shit':", similar_words)

Vector for 'shit': [ 0.59024054  0.44312063  0.26723507 -0.03198092  0.44472104 -0.10116369
  0.89917904  1.0842729  -0.5306927  -0.4790301   0.3180201  -0.792939
  0.06238808 -0.01981799 -0.24417745 -0.20351759  0.60264194 -0.24241944
 -0.01383593 -0.6195277   0.281818    0.51503664  0.06008047 -0.5408794
  0.00248451  0.6456495  -0.2748829  -0.203345    0.02977721 -0.16061579
 -0.20355712 -0.39545465  0.1350985   0.1073214  -0.06123455  0.13844776
  0.31111005  0.10492787 -0.33643192 -0.3703423  -0.04687409 -0.17029992
  0.02429629  0.00369885  0.5865334  -0.16098954 -0.45374376  0.21585917
  0.24858278  0.6835139   0.05257368 -0.09547304 -0.27879342  0.2589546
 -0.17011829  0.4053065  -0.37350973  0.35203427 -0.17029418  0.22404604
 -0.36951676 -0.4289252   0.08512682 -0.29306898 -0.5937988   0.2071809
  0.69880307  0.3374603  -0.27806512  0.4828797  -0.09829509  0.12977555
  0.27532887 -0.23000939  0.1043769  -0.23460226  0.51300955  0.20568602
 -0.5910166  -0.41770282 -0.6940532  

In [10]:

# Check most similar words
word = "hate"
similar_words = model.wv.most_similar(word, topn=5)
print(f"Words similar to '{word}': {similar_words}")


Words similar to 'hate': [('hates', 0.6982182860374451), ('fault', 0.6954658627510071), ('tew', 0.6827342510223389), ('pfp', 0.6811203360557556), ('istg', 0.6807213425636292)]


In [11]:
from gensim.models import FastText

model = FastText(vector_size=100, window=5, min_count=1, sg=1, epochs=10)
model.build_vocab(tokenized_sentences)
model.train(tokenized_sentences, total_examples=len(tokenized_sentences), epochs=10)

model.save("fasttext_model.model")

In [12]:
model = FastText.load("fasttext_model.model")

word_vector = model.wv['hate']
print("Vector for 'hate':", word_vector)

similar_words = model.wv.most_similar('hate')
print("Similar words:", similar_words)

Vector for 'hate': [-2.48300359e-01  1.09273024e-01 -6.71660364e-01  2.27219760e-01
  5.62640250e-01  4.35334861e-01 -3.13606173e-01  1.45046905e-01
  8.83883834e-01 -9.38965857e-01 -5.20260274e-01 -6.16742909e-01
  1.44336507e-01  9.28647995e-01  2.25819033e-02 -2.20994428e-01
  5.68438768e-01  5.45261726e-02 -1.63975745e-01 -1.06441414e+00
  5.85953966e-02  8.31971586e-01  5.65044768e-02 -2.93443680e-01
 -6.65605485e-01 -5.01947463e-01  5.17006874e-01 -1.88412443e-02
  1.31390961e-02  5.62422037e-01  8.77174914e-01  3.13685179e-01
  2.81409472e-01 -2.45108113e-01  8.27184618e-02  1.84413284e-01
 -2.70475745e-01  7.68536210e-01 -3.24853063e-01 -6.84299786e-03
 -7.70946443e-01  4.64807063e-01  9.36575606e-02 -5.06642580e-01
 -5.20099342e-01 -3.12623739e-01 -5.40407717e-01  5.22264242e-01
  2.42474422e-01 -1.88420102e-01  2.39965487e-02 -2.40830854e-01
 -3.88183519e-02  1.61293417e-03  5.61385788e-02 -2.33369589e-01
 -1.17531925e-01 -3.83623280e-02  3.10898662e-01  1.84851035e-01
  3.99

In [13]:
import numpy as np
from gensim.models import FastText
from scipy.sparse import csr_matrix
from collections import defaultdict

# Load FastText model
model = FastText.load("fasttext_model.model")

# Sample sentences
sentences = data['clean_tweet_2']
# Ensure all sentences are strings and handle NaN values
sentences = [str(sentence) for sentence in sentences if pd.notnull(sentence)]

# Create vocabulary
vocab = list(model.wv.index_to_key)
# Create a word-to-index dictionary for faster lookups
word_to_index = defaultdict(lambda: -1, {word: i for i, word in enumerate(vocab)})

# Create embedding matrix (X) - word embeddings
embedding_dim = model.vector_size
X = np.zeros((len(vocab), embedding_dim), dtype=np.float32)
print("Shape of X:", X.shape)
for i, word in enumerate(vocab):
    X[i] = model.wv[word]

# Create hypergraph incidence matrix (H)
num_sentences = len(sentences)
H = np.zeros((len(vocab), num_sentences), dtype=np.float32)

print("Shape of H:", H.shape)

for j, sentence in enumerate(sentences):
    for word in sentence.split():
        i = word_to_index[word]
        if i != -1:
            H[i, j] = 1

# Convert to sparse matrices
X_sparse = csr_matrix(X)  # Feature matrix
H_sparse = csr_matrix(H)  # Hypergraph incidence matrix

print("Feature Matrix Shape (X):", X_sparse.shape)
print("Incidence Matrix Shape (H):", H_sparse.shape)


Shape of X: (60373, 100)
Shape of H: (60373, 149822)
Feature Matrix Shape (X): (60373, 100)
Incidence Matrix Shape (H): (60373, 149822)


In [14]:
!pip install dhg

Defaulting to user installation because normal site-packages is not writeable
  Using cached torch-1.13.1-cp39-none-macosx_11_0_arm64.whl.metadata (23 kB)
Using cached torch-1.13.1-cp39-none-macosx_11_0_arm64.whl (53.2 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.5.1
    Uninstalling torch-2.5.1:
      Successfully uninstalled torch-2.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lightning 2.5.0.post0 requires torch<4.0,>=2.1.0, but you have torch 1.13.1 which is incompatible.
pytorch-lightning 2.5.0.post0 requires torch>=2.1.0, but you have torch 1.13.1 which is incompatible.
torchaudio 2.5.1 requires torch==2.5.1, but you have torch 1.13.1 which is incompatible.
torchmetrics 1.6.1 requires torch>=2.0.0, but you have torch 1.13.1 which is incompatible.
torchvision 0.20.1 requires torch==2.5.1, but you have torch 1.13.1 which

In [15]:
import dhg
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.nn.functional import one_hot
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

/Users/admin/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:

vocab = list(model.wv.index_to_key)

from collections import defaultdict
word_to_index = defaultdict(lambda: -1, {word: idx for idx, word in enumerate(vocab)})

In [17]:
hyperedges = []
for sentence in sentences:
    word_indices = list({word_to_index[word] for word in sentence.split() if word_to_index[word] != -1})
    hyperedges.append(word_indices)

print(f"Number of hyperedges (tweets): {len(hyperedges)}")

Number of hyperedges (tweets): 149822


In [18]:
G = dhg.Hypergraph(num_v=len(vocab), e_list=hyperedges)

In [19]:
import numpy as np


embedding_dim = model.vector_size


X = np.zeros((len(vocab), embedding_dim), dtype=np.float32)


for i, word in enumerate(vocab):
    X[i] = model.wv[word]

print("Feature matrix X shape:", X.shape)

Feature matrix X shape: (60373, 100)


In [20]:
num_edges = len(hyperedges)  # each tweet is a hyperedge

train_idx, test_idx = train_test_split(
    np.arange(num_edges),
    test_size=0.2,
    stratify=y,
    random_state=42
)

In [21]:
train_idx = torch.tensor(train_idx, dtype=torch.long)
test_idx = torch.tensor(test_idx, dtype=torch.long)
y = torch.tensor(y, dtype=torch.long)

In [22]:
X_tensor = torch.tensor(X, dtype=torch.float)
print(X_tensor.shape[1])

100


In [23]:
from dhg import Hypergraph
hg = Hypergraph(len(vocab), hyperedges)

In [24]:
!pip install -U dhg


Defaulting to user installation because normal site-packages is not writeable


In [25]:
from dhg.models import HGNN

In [26]:
import torch.nn as nn
import torch.nn.functional as F
from dhg.models import HGNN

class MyHGNN(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim):
        super().__init__()
        self.hgnn = HGNN(in_dim, hidden_dim, out_dim)

    def forward(self, x, hg):
        return self.hgnn(x, hg)

in_dim = X_tensor.shape[1]     # embedding dimension (100 for FastText)
hidden_dim = 64
out_dim = 2                    # binary classification: hate or not

model = MyHGNN(in_dim, hidden_dim, out_dim)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()


In [27]:
if isinstance(y, list):
    y = torch.tensor(y, dtype=torch.long)

In [28]:
print("Type of hyperedges:", type(hyperedges))
print("Length of hyperedges:", len(hyperedges))

Type of hyperedges: <class 'list'>
Length of hyperedges: 149822


In [1]:
y.shape

NameError: name 'y' is not defined

In [30]:
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    out = model(X_tensor, hg)

    # Aggregate node predictions to edge level (sentence-level)
    edge_out = torch.zeros((len(hyperedges), out.shape[1]))

    for e, nodes in enumerate(hyperedges):
        if len(nodes) > 0:
            edge_out[e] = out[list(nodes)].mean(dim=0)


    loss = criterion(edge_out[train_idx], y[train_idx])
    loss.backward()
    optimizer.step()


    model.eval()
    with torch.no_grad():
        preds = edge_out[test_idx].argmax(dim=1)
        acc = (preds == y[test_idx]).float().mean().item()
    print(f"Epoch {epoch+1}: Loss = {loss.item():.4f}, Test Accuracy = {acc:.4f}")




KeyboardInterrupt: 

In [ ]:
def predict_sentence(sentence, model, ft_model, vocab, X_tensor):
    model.eval()
    words = sentence.lower().split()

    node_indices = [vocab[word] for word in words if word in vocab]

    if not node_indices:
        return "No known words in sentence."


    with torch.no_grad():
        out = model(X_tensor, hg)  # shape: (num_nodes, out_dim)
        sentence_embed = out[node_indices].mean(dim=0)  # average over node predictions
        prediction = torch.argmax(sentence_embed).item()
        probs = torch.softmax(sentence_embed, dim=0)

    label = "Hate Speech" if prediction == 1 else "Not Hate Speech"
    confidence = probs[prediction].item()

    return f"Prediction: {label} (confidence: {confidence:.4f})"


In [ ]:
ft_model = FastText.load("fasttext_model.model")

In [ ]:
vocab_dict = {word: idx for idx, word in enumerate(vocab)}


In [ ]:
result = predict_sentence("nigga i hate tests", model, ft_model, vocab_dict, X_tensor)
print(result)


Prediction: Not Hate Speech (confidence: 0.7355)


In [ ]:
print("out shape:", out.shape)  # should be (num_nodes, out_dim)
print("Number of hyperedges:", len(hyperedges))
print("Sample hyperedge nodes:", hyperedges[0])

print("edge_out type:", type(edge_out))
print("edge_out shape:", edge_out.shape if isinstance(edge_out, torch.Tensor) else "not tensor")
print("train_idx max:", train_idx.max().item(), "| len(train_idx):", len(train_idx))
print("y shape:", y.shape, "| y dtype:", y.dtype)


out shape: torch.Size([60373, 2])
Number of hyperedges: 149822
Sample hyperedge nodes: (0,)
edge_out type: <class 'torch.Tensor'>
edge_out shape: torch.Size([149822, 2])
train_idx max: 149821 | len(train_idx): 119857
y shape: torch.Size([149822]) | y dtype: torch.int64


In [ ]:
empty_count = sum([1 for e in hyperedges if len(e) == 0])
print(f"Empty hyperedges: {empty_count} / {len(hyperedges)}")


Empty hyperedges: 14 / 149822


In [ ]:
torch.save(model.state_dict(), "model_weights.pth")